In [1]:
from sklearn.preprocessing import KBinsDiscretizer

In [2]:
experiments = [
    [0, 100, 1, 4, "Q"],
    [0, 50, 1, 4, "Q"],
    [0, 100, 1, 3, "Q"],
    [0, 50, 1, 3, "Q"],
    [0, 10, 1, 3, "Q"],
    [0, 100, 1, 4, "U"],
    [0, 50, 1, 4, "U"],
    [0, 100, 1, 3, "U"],
    [0, 50, 1, 3, "U"],
    [0, 10, 1, 3, "U"],
]

In [3]:
with open("datasets/tests.txt", "w") as file:
    file.write("#\n")
    file.write("# from, to, step, #bins, Q/U\n")
    file.write("# discretized data\n")
    file.write("# cut points\n")
    file.write("#\n")
    for experiment in experiments:
        (from_, to_, step_, bins_, strategy) = experiment
        disc = KBinsDiscretizer(n_bins=bins_, encode='ordinal', strategy='quantile' if strategy.strip() == "Q" else 'uniform')
        data = [[x] for x in range(from_, to_, step_)]
        disc.fit(data)
        result = disc.transform(data)
        file.write(f"{from_}, {to_}, {step_}, {bins_}, {strategy}\n")
        sep = ""
        for res in result:
            file.write(f"{sep}{int(res):d}")
            sep= ", "
        file.write("\n")
        sep = ""
        for res in disc.bin_edges_[0]:
            file.write(sep + str(round(res,5)))
            sep = ", "
        file.write("\n")